# 准备工作

## 前序数据预处理

In [15]:
#!/usr/bin/env python
# coding: utf-8

# 批量处理文件夹中所有文件
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts

    
# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )

    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')

    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)




#导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''


# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50


# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)


split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")



print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")



split_docs[300].page_content



['../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.20.80.02）告警处理.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.20.80.02）故障管理概述.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（数据业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（端到端业务）.pdf', '../data_base/knowledge_path/VMAX-S\\ZXVMAX-S（V6.23）产品描述（语音业务）.pdf']
切分后的文件数量：891
切分后的字符数（可以用来大致评估 token 数）：319269


"ZXVMAX-S多维价值分析系统告警处理\n4.修改配置项'Agent的Java堆栈大小'的参数值，重启已修正的flume角色实例；\n5.等待5分钟，检查告警是否已恢复。如果恢复则结束，如果没恢复则进行第6步；\n6.以root用户登录产生该告警的服务器；\n7.执行ps-ef|grepFlumeAgent命令，获取flume服务（FlumeAgent进程）的PID；\n8.执行jmap-heap<PID>命令，检查flume服务（FlumeAgent进程）'Agent的Java堆栈大小'配置是否生效（蓝色字体的值）。如果不生效则跳到第10步，如果生效则进行第9步；\n9.等待5分钟，检查告警是否已恢复。如果恢复则结束，如果没恢复则进行第10步；\n10.备案，等待5分钟，执行步骤8，查看flume进程的堆内存配置是否生效。生效则结束，不生效则通知上级维护人员进一步排查。\n1.238124010036FlumeAgent进程5分钟意外退出次数告警描述"

## 前序embedding模型

In [16]:
# ZhipuAIEmbeddings模型
# import os
# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())

# from langchain_community.embeddings import ZhipuAIEmbeddings

# my_emb = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key = os.environ['ZHIPUAI_API_KEY'],
# )

In [17]:
from langchain_ollama.embeddings import OllamaEmbeddings

# 初始化嵌入模型
my_emb = OllamaEmbeddings(
    base_url='http://localhost:11434',
    model="bge-m3:latest"
)

# 数据库选型：Chroma

## 构建Chroma向量库

涉及到的API

In [18]:
# langchain_chroma.vectordbs.Chroma
# class langchain_chroma.vectordbs.Chroma(
#     collection_name: str = 'langchain', 
# embedding_function: Embeddings | None = None, 
# persist_directory: str | None = None, 
# client_settings: Settings | None = None, 
# collection_metadata: Dict | None = None, 
# client: ClientAPI | None = None, 
# relevance_score_fn: Callable[[float], float] | None = None, 
# create_collection_if_not_exists: bool | None = True)


# classmethod from_documents(
#     documents: List[Document], 
# embedding: Embeddings | None = None, 
# ids: List[str] | None = None, 
# collection_name: str = 'langchain', 
# persist_directory: str | None = None, 
# client_settings: Settings | None = None, 
# client: ClientAPI | None = None, 
# collection_metadata: Dict | None = None, 
# **kwargs: Any)

直接调用Chroma.from_documents (推荐)

In [19]:
# 汇总

import os
from langchain_community.vectorstores import Chroma
# from langchain_chroma.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings

# 定义持久化目录
persist_directory = '../data_base/vector_db/chroma-vmax'

# 创建嵌入模型
my_emb = OllamaEmbeddings(
    base_url='http://localhost:11434',
    model="bge-m3:latest"
)

try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[0:10],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        # documents=split_docs,  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=my_emb,
        # collection_name="test1", # 如果不指定默认为langchain
        persist_directory=persist_directory, # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

向量数据库已成功持久化到磁盘。


C:\Users\will\AppData\Local\Temp\ipykernel_2896\2982754907.py:28: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


以上代码会报错： 持久化过程中发生错误: Error code: 400, with error text {"error":{"code":"1214","message":"input数组最大不得超过64条"}}

建议分批次导入

In [ ]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings

# 定义持久化目录
persist_directory = '../data_base/vector_db/chroma-vmax'

# 创建嵌入模型
my_emb = OllamaEmbeddings(
    base_url='http://localhost:11434',
    model="bge-m3:latest"
)

# 定义每批处理的文档数量
batch_size = 30

split_docs=split_docs[:100]

try:
    # 计算总批次数
    total_batches = (len(split_docs) + batch_size - 1) // batch_size
    
    # 初始化向量数据库（如果是第一次创建）
    vectordb = None
    
    for batch_num in range(total_batches):
        # 计算当前批次的起始和结束索引
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(split_docs))
        
        # 获取当前批次的文档
        batch_docs = split_docs[start_idx:end_idx]
        
        print(f"正在处理第 {batch_num + 1}/{total_batches} 批文档 (文档 {start_idx}-{end_idx-1})")
        
        if batch_num == 0:
            # 第一次创建向量数据库
            vectordb = Chroma.from_documents(
                collection_name='vmaxs',
                documents=batch_docs,
                embedding=my_emb,
                persist_directory=persist_directory
            )
        else:
            # 后续批次添加到现有集合
            vectordb.add_documents(batch_docs)
        
        # 每批处理后持久化
        vectordb.persist()
        print(f"第 {batch_num + 1} 批文档已成功导入并持久化")
    
    print("所有文档已成功导入并持久化到向量数据库。")
    
except Exception as e:
    print(f"处理过程中发生错误: {e}")

也可以参考数据库官方文档，进行操作： https://docs.trychroma.com/docs/overview/introduction

## 连接Chroma数据库

In [ ]:
# 数据库路径 ./chroma2

In [1]:
import chromadb
chroma_client  = chromadb.PersistentClient(path="../data_base/vector_db/chroma-vmax")

In [2]:
chroma_client.list_collections()

['langchain', 'vmax-s']

In [7]:
# 获取一个存在的Collection对象
collection = chroma_client.get_collection("langchain")

In [8]:
collection.count()  #  returns the number of items in the collection.

10

In [9]:
# collection.peek()  # returns a list of the first 10 items in the collection.

In [10]:
# 获取所有数据（默认不返回嵌入向量）
all_data = collection.get()
print(all_data)


{'ids': ['892b501d-248d-4b96-9fec-64f2efab9f86', '2eb6c129-4d1f-4eed-a5c7-bc5e2e51a6ce', '459ef97b-04a3-4f0b-909b-404e7e955738', '2c67d77b-d2a4-4f0d-85e7-1bf5ccabd016', 'df6c4497-9a83-4bff-ba7d-025ffd5290ac', '0d1ffdf7-c6cb-4866-8363-b87d50390e45', 'a6699ba4-b2d5-4a8a-a62b-5dc5828486ac', 'fcea22f0-3b56-436a-b976-be43e68f0acc', 'f07074ea-db04-4d4a-8a45-902b3ca108fc', '00364fd3-9444-42c4-865c-0dc5f83b4ba9'], 'embeddings': None, 'documents': ['ZXVMAX-S\n多维价值分析系统告警处理产品版本：V6.20.80.02\n中兴通讯股份有限公司地址：深圳市南山区高新技术产业园科技南路中兴通讯大厦邮编：518057\n电话：0755-26770800\n\xa0\xa0\xa0400-8301118\n\xa0\xa0\xa0\xa0\xa0\xa0\n800-8301118（座机）技术支持网站：http://support.zte.com.cn电子邮件：800@zte.com.cn', '法律声明本资料著作权属中兴通讯股份有限公司所有。未经著作权人书面许可，任何单位或个人不得以任何方式摘录、复制或翻译。侵权必究。\xa0和\xa0是中兴通讯股份有限公司的注册商标。中兴通讯产品的名称和标志是中兴通讯的专有标志或注册商标。在本手册中提及的其他产品或公司的名称可能是其各自所有者的商标或商名。在未经中兴通讯或第三方商标或商名所有者事先书面同意的情况下，本手册不以任何方式授予阅读者任何使用本手册上出现的任何标记的许可或权利。本产品符合关于环境保护和人身安全方面的设计要求，产品的存放、使用和弃置应遵照产品手册、相关合同或相关国法律、法规的要求进行。如果本产品进行改进或技术变更，恕不另行专门通知。当出现产品改进或者技术变更时，您可以通过中兴通讯技术

## 向量检索
### 3.1 相似度检索
Chroma的相似度搜索使用的是余弦距离，即：
$$
similarity = cos(A, B) = \frac{A \cdot B}{\parallel A \parallel \parallel B \parallel} = \frac{\sum_1^n a_i b_i}{\sqrt{\sum_1^n a_i^2}\sqrt{\sum_1^n b_i^2}}
$$
其中$a_i$、$b_i$分别是向量$A$、$B$的分量。

当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用`similarity_search`函数。

使用Chroma.from_documents时，用这个方法： 

In [20]:
question="vmax"

In [21]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [22]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
ZXVMAX-S
多维价值分析系统告警处理产品版本：V6.20.80.02
中兴通讯股份有限公司地址：深圳市南山区高新技术产业园科技南路中兴通讯大厦邮编：518057
电话：0755-26770800
   400-8301118
      
800-8301118（座机）技术支持网站：http://support.zte.com.cn电子邮件：800@zte.com.cn
--------------
检索到的第1个内容: 
ZXVMAX-S
多维价值分析系统告警处理产品版本：V6.20.80.02
中兴通讯股份有限公司地址：深圳市南山区高新技术产业园科技南路中兴通讯大厦邮编：518057
电话：0755-26770800
   400-8301118
      
800-8301118（座机）技术支持网站：http://support.zte.com.cn电子邮件：800@zte.com.cn
--------------
检索到的第2个内容: 
1.14141001HBase元数据不一致...........................................15
1.15141001localhbase元数据异常.........................................15
1.16180000Master状态切换告警..........................................
--------------


### 3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [23]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [24]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
ZXVMAX-S
多维价值分析系统告警处理产品版本：V6.20.80.02
中兴通讯股份有限公司地址：深圳市南山区高新技术产业园科技南路中兴通讯大厦邮编：518057
电话：0755-26770800
   400-8301118
      
800-8301118（座机）技术支持网站：http://support.zte.com.cn电子邮件：800@zte.com.cn
--------------
MMR 检索到的第1个内容: 
1.33111010028Zookeeper进程事务日志目录可用空间大小......................25
1.34111010029Zookeeper进程客户端最大请求时延占比超时时长................26
1.35111010030Zookeeper进程5分钟意外退出次数.............................27
1.36111010037Zoo
--------------
MMR 检索到的第2个内容: 
R1.0
2022-06-20
第一次发布资料编号∶SJ-20220623151803-011
发布日期∶2022-06-20（R1.0）
--------------


# 数据库选型：Milvus

## 构建Milvus向量库

In [ ]:
import os
from langchain_community.vectorstores import Milvus
from langchain_core.documents import Document


# 创建嵌入模型
my_emb = OllamaEmbeddings(
    base_url='http://localhost:11434',
    model="bge-m3:latest"
)

# 向量库创建
connection_args = {
    "host": "192.168.0.188",
    "port": "19530",
}


# 定义每批处理的文档数量
batch_size = 30

# 如果只想导入部分数据
# split_docs = split_docs[:3]
try:
    # 计算总批次数
    total_batches = (len(split_docs) + batch_size - 1) // batch_size
    
    # 初始化向量数据库（如果是第一次创建）
    vectordb = None
    
    for batch_num in range(total_batches):
        # 计算当前批次的起始和结束索引
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(split_docs))
        
        # 获取当前批次的文档
        batch_docs = split_docs[start_idx:end_idx]
        
        print(f"正在处理第 {batch_num + 1}/{total_batches} 批文档 (文档 {start_idx}-{end_idx-1})")

        if batch_num == 0:
            # 第一次创建向量数据库
            vectordb = Milvus.from_documents(
            documents=batch_docs,
            embedding=my_emb,
            collection_name="vmaxs",
            drop_old=False,
            connection_args=connection_args,
            )

        else:
            # 后续批次添加到现有集合
            vectordb.add_documents(batch_docs)
        
        # 每批处理后持久化
        # vectordb.persist()
        print(f"第 {batch_num + 1} 批文档已成功导入并持久化")
    
    print("所有文档已成功导入并持久化到向量数据库。")
    
except Exception as e:
    print(f"处理过程中发生错误: {e}")

In [26]:
from langchain_community.vectorstores import Milvus

my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )




C:\Users\will\AppData\Local\Temp\ipykernel_2896\155588456.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(
Failed to create new connection using: 6035000a1b3241f0af07ef903e49f4c4


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 192.168.0.188:19530, illegal connection params or server unavailable)>